In [1]:
import os
from pathlib import Path

os.chdir(Path(os.getcwd()).parents[2])
os.getcwd()

target = "mth" # mth, pce

In [2]:
import torch
import torch.nn as nn
import numpy as np
import kaleido
from torch.utils.data import DataLoader
from data.perovskite_dataset import (
    PerovskiteDataset1d,
    PerovskiteDataset2d,
    PerovskiteDataset3d,
    PerovskiteDataset2d_time,
)
from models.resnet import ResNet152, ResNet, BasicBlock, Bottleneck
from base_model import seed_worker
from data.augmentations.perov_1d import normalize
from os.path import join

import dice_ml
from dice_ml import Dice

data_dir = "/home/l727n/Projects/Applied Projects/ml_perovskite/preprocessed"

if target == "pce":
    checkpoint_dir = "/home/l727n/E132-Projekte/Projects/Helmholtz_Imaging_ACVL/KIT-FZJ_2021_Perovskite/data_Jan_2022/checkpoints"

    path_to_checkpoint = join(
        checkpoint_dir, "1D-epoch=999-val_MAE=0.000-train_MAE=0.490.ckpt"
    )
else:
    checkpoint_dir = "/home/l727n/E132-Projekte/Projects/Helmholtz_Imaging_ACVL/KIT-FZJ_2021_Perovskite/data_Jan_2022/mT_checkpoints/checkpoints"

    path_to_checkpoint = join(
        checkpoint_dir, "mT_1D_RN152_full-epoch=999-val_MAE=0.000-train_MAE=40.332.ckpt"
    )


/home/l727n/miniconda3/envs/perovskite/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Import of model and computation of counterfactuals (higher and lower than response confidence intervall)

## PyTorch 1D Model

In [3]:
#### 1D Model

hypparams = {
    "dataset": "Perov_1d",
    "dims": 1,
    "bottleneck": False,
    "name": "ResNet152",
    "data_dir": data_dir,
    "no_border": False,
    "resnet_dropout": 0.0,
    "stochastic_depth": 0.0,
    "norm_target": True if target == "pce" else False,
    "target": "PCE_mean" if target == "pce" else "meanThickness"
}

model = ResNet.load_from_checkpoint(
    path_to_checkpoint,
    block=BasicBlock,
    num_blocks=[4, 13, 55, 4],
    num_classes=1,
    hypparams=hypparams,
)

model.eval()
print("Loaded")

import pandas as pd

train_set = PerovskiteDataset1d(
    data_dir,
    transform=normalize(model.train_mean, model.train_std),
    scaler=model.scaler,
    no_border=False,
    return_unscaled=False if target == "pce" else True,
    label="PCE_mean" if target == "pce" else "meanThickness",
)

trainloader_full = DataLoader(
    train_set,
    batch_size=len(train_set),
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    worker_init_fn=seed_worker,
    persistent_workers=True,
)

timeseries, label = next(iter(trainloader_full))
data = timeseries.numpy().reshape(-1, 4 * 719)
data = pd.DataFrame(data)

label = pd.DataFrame(label.numpy())
data["target"] = label

feature_names = data.columns[0:-1].to_list()


data_dice = dice_ml.Data(
    dataframe=data, continuous_features=feature_names, outcome_name="target"
)

tensor([0.2697, 0.0191, 0.0057, 0.0216]) tensor([0.1589, 0.0106, 0.0030, 0.0145])
Loaded


In [4]:
class Wrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, image):
        x = torch.Tensor(image.values)
        x = image.reshape(-1, 4, 719)
        y_pred = self.model(x)

        return y_pred.detach().numpy()

    def predict(self, image):
        x = torch.Tensor(image.values)
        x = x.reshape(-1, 4, 719)
        y_pred = self.model(x)
        y_pred = y_pred.detach().squeeze(1).numpy()

        return y_pred


In [5]:
import dice_ml
from dice_ml.utils import helpers

wModel = Wrapper(model)

model_dice = dice_ml.Model(model=wModel, backend="sklearn", model_type="regressor")

cf_methode = "genetic"
methode = Dice(data_dice, model_dice, method=cf_methode)


In [20]:
n = 4

query_instances = pd.DataFrame(data.iloc[n][0:-1]).T
genetic = methode.generate_counterfactuals(
    query_instances,
    total_CFs=1,
    desired_range=[0.9, 100] if target == "pce" else [1300, 5000],
)

cf_high = np.array(genetic.cf_examples_list[0].final_cfs_df)[:, 0:-1].reshape(4, 719)
x = np.array(data.iloc[n][0:-1]).reshape(4, 719)

if target == "pce":
    scaler = PerovskiteDataset1d(
        data_dir=data_dir,
        transform=normalize(model.train_mean, model.train_std),
        fold=None,
        split="train",
        no_border=False,
        return_unscaled= False if target == "pce" else True,
        label="PCE_mean" if target == "pce" else "meanThickness",
        val=False,
        ).get_fitted_scaler()

    y = np.round(scaler.inverse_transform(data.iloc[n]["target"].reshape([-1, 1])), 2)
    y_cf_high = np.round(
        scaler.inverse_transform(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1])
        ),
        2,
    )
else:
    y = np.round(data.iloc[n]["target"].reshape([-1, 1]), 2)
    y_cf_high = np.round(
        np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1]), 2,
    )


100%|██████████| 1/1 [00:33<00:00, 33.53s/it]


In [26]:
genetic = methode.generate_counterfactuals(
    query_instances, total_CFs=1, desired_range=[-100, -0.9] if target == "pce" else [0, 700]
)

cf_low = np.array(genetic.cf_examples_list[0].final_cfs_df)[:, 0:-1].reshape(4, 719)

if target == "pce":
    y_cf_low = np.round(
        scaler.inverse_transform(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1])
        ),
        2,
    )
else:
    y_cf_low = np.round(
            np.array(genetic.cf_examples_list[0].final_cfs_df)[:, -1].reshape([-1, 1]),
        2,
    )

print("High: ", y_cf_high, " Low: ",y_cf_low, " True: ",y)

100%|██████████| 1/1 [00:33<00:00, 33.87s/it]

High:  [[1334.64]]  Low:  [[661.54]]  True:  [[765.48]]


In [27]:
import plotly.graph_objects as go


def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f"<b>{title}</b>"
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f"{title}<br>{subtitle}"

color = ["#E1462C", "#0059A0", "#5F3893", "#FF8777","#0A2C6E", "#CEDEEB"]

from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2,
    cols=4,
)

fig.add_trace(
    go.Scatter(y=x[0], name="ND", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(y=cf_high[0], name="cf ND", marker_color=color[1], showlegend=False, line=dict(width=2.5)),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=x[1], name="LP725", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(y=cf_high[1], name="cf LP725", marker_color=color[2], showlegend=False, line=dict(width=2.5)),
    row=1,
    col=2,
)
fig.add_trace(
    go.Scatter(
        y=x[2], name="LP780", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(y=cf_high[2], name="cf LP780", marker_color=color[0], showlegend=False, line=dict(width=2.5)),
    row=1,
    col=3,
)
fig.add_trace(
    go.Scatter(
        y=x[3], name="SP775", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=4,
)
fig.add_trace(
    go.Scatter(
        y=cf_high[3], name="cf NSP775D", marker_color=color[3], showlegend=False, line=dict(width=2.5)
    ),
    row=1,
    col=4,
)

fig.add_trace(
    go.Scatter(y=x[0], name="ND", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(y=cf_low[0], name="cf ND", marker_color=color[1], showlegend=False, line=dict(width=2.5)),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        y=x[1], name="LP725", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(y=cf_low[1], name="cf LP725", marker_color=color[2], showlegend=False, line=dict(width=2.5)),
    row=2,
    col=2,
)
fig.add_trace(
    go.Scatter(
        y=x[2], name="LP780", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=3,
)
fig.add_trace(
    go.Scatter(y=cf_low[2], name="cf LP780", marker_color=color[0], showlegend=False, line=dict(width=2.5)),
    row=2,
    col=3,
)
fig.add_trace(
    go.Scatter(
        y=x[3], name="SP775", marker_color="grey", opacity=0.3, showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=4,
)
fig.add_trace(
    go.Scatter(
        y=cf_low[3], name="cf NSP775D", marker_color=color[3], showlegend=False, line=dict(width=2.5)
    ),
    row=2,
    col=4,
)

fig.update_yaxes(title=None,zeroline=False, showticklabels = False,)
fig.update_yaxes(title_text=None, range = [-2.1,4.1],  tickfont= dict(size=18, family="Helvetica", color="rgb(0,0,0)"))
fig.update_xaxes(title=None, zeroline=False,  tickfont= dict(size=18, family="Helvetica", color="rgb(0,0,0)"))

fig.update_yaxes(title="Intensity",showticklabels = True, row=1, col=1)
fig.update_yaxes(showticklabels = True, row=2, col=1)
fig.update_xaxes(title="Timesteps", row=2, col=1)

fig.update_layout(
    legend_title=None,
    title_y=0.95,
    title_x=0.035,
    template="plotly_white",
    height=500,
    width=1200,
)

fig.write_image("xai/images/"+ target + "/1D/1D_cf_nn.png", scale=2)

fig.show()
